# 0. Installation and import the libraries

FIRST THING FIRST. 

Install sylfenUtils==1.14 in a virtual environment. Replace sylfen-user by your own account on Gaston in the following command and enter your password when asked:
```
pip install sylfenUtils @git+ssh://<sylfen-user>@cheylas.sylfen.com/home/<sylfen-user>/gitRepos/sylfenUtils.git@1.14
```
In order to run your jupyter-notebook with your virtual environment follow instructions [here](https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove).
Then import the libraries.

In [ ]:
from sylfenUtils import comUtils
import os,sys,re, pandas as pd

Check that you have installed the right version of sylfenUtils.

# 1.Build the configuration of your project using the Conf_generator
For convenience put your configuration work in a conf.py file which should serve as an object with all
the important information.

## 1.1 Configure your project and call the Conf_generator class

Let's say you have a modbus table of a device in a "modbus_dummy.csv"(download it [here](wiki.sylfen.com/notebooks/data/modbus_dummy.csv) ) file and you can call this device "dummy". Read it with pandas without forgetting to adjust the path of the file depending where it is located on your device. 

In [ ]:
dummy_modbus_map=pd.read_csv('data/modbus_dummy.csv',index_col=0)
dummy_modbus_map

To use the dumper class of the comUtils module of the sylfenUtils library you need to build a df_plc dataframe with at least following columns:
- TAGS(as index)
- DESCRIPTION
- UNITE
- DATATYPE. 
For example you can build it from your dummy_modbus_device as follow:

In [ ]:
df_plc=dummy_modbus_map[['description','unit','type']]
df_plc.columns=['DESCRIPTION','UNITE','DATATYPE']
dummy_df_plc=df_plc

If you create a fonction that returns a dictionnary of all the objects of your configuration, the Conf_generator class will save that dictionnary as a .pkl file and will rerun the function if the pkl does not exist or on demand otherwise it will just load the dictionnary. If your configuration work takes time it will enable you to save a lot of loading time when executing your programm or tests. Instanciate your configuration object as follow.     

In [ ]:
from sylfenUtils import Conf_generator
def generate_dummy_conf():
    return {
        'dummy_modbus_map':dummy_modbus_map,
        'dummy_df_plc':dummy_df_plc,
    }
dummy_conf=Conf_generator.Conf_generator('dummy_project',generate_dummy_conf)

It as created a folder in the home of the user executing the script with following content : 

In [ ]:
os.listdir(dummy_conf.project_folder)

- **dummy_project_daily** would contain(by default) the data that are going to be generated => dummy_conf.FOLDERPKL
- **conf_dummy_project.pkl** is the object containing all the important information of the project(the configuration) saved as dictionnary. The keys are then attributes of dummy_conf object.  
- **log** : dummy_conf.LOG_FOLDER which is the folder where the logs are going to be stored (by default)
- **parameters.conf** : is a standard file with default settings that you are able to to modify.  

Your configuration is done now. 

**Note**: You can also create your own class to generate the configuration that inheritates from Conf_generator class.

## 1.2 Modify default settings if you need it

As said in previous section you can change the default settings at any time.
For the moment the content of the **parameters.conf** file is :  

In [ ]:
with open(dummy_conf.file_parameters,'r') as f : 
    for l in f.readlines():
        print(l)

Explanation of the parameters:
- **database parameters**:For the data acquisition you NEED TO INSTALL POSTGRESSQL and CREATE A DATABASE WITH A USER having permissions to create databases. Follow instructions [here](http://wiki.sylfen.com/sql/)
  - db_host : host of the postgres database
  - db_port : port of the postgres database
  - db_name : name of the database.
  - db_user : valid postgressql user with
  - db_password : db_user password to connect to the database
  - DB_TABLE : database table where the data are going to be stored with timestamp,tag,value columns.
- **other shared variables**:  
  - PARKING_TIME:the size of the database in seconds until it is emptied.
  - TZ_RECORD: the timezone used to park the data. By default it is CET whereas it would be better to have UTC. But for people living in CET timezone it is easier to open a day-folder with all the hours  of the current day(and not having to open 2 folders for that).
  - SIMULATOR:whether the simulator should be used instead of connecting to the real devices(convenient when working offline or not on the private network where the devices are available).
  - FOLDERPKL:path of the folder where the daily parked data are stored. If *default* the data are found in dummy_conf.project_folder.
  - LOG_FOLDER:path of the folder where the logs are found.   
Precise where the data will be stored on your machine, which timezone, how big the database will be and the logs folder.
- **dashboard specific variables**
  - TEST_ENV:if True the *dashboard.py* file will be run on port 15000 from flask(not wsgi production environment).
  - TMAX: works only if TEST_ENV is true. The end datetime of datetimepicker of the the dashboard.  

These parameters are then accessible from any instance of Conf_generator (or your own children class) as soon as with a name of the project. As such all the other higher-level objects will know these information.

# 2. DUMP DATA 
You learn here how to istanciate your dumper to record data as daily parked format. 

## 2.1 Create modbus devices, add them to your DUMPER which will record data as parked format
Your first device is the dummy device whose ip adress, byte order, word order are known. You just need to enter 
- ip of the device
- running port
- the name of the device(you can choose any, this has no effect)
- the plc dataframe *dfplc*
- the modbus map 
- the words(*wo*) and bytes(*bo*) endianess. Only 'little' or 'big' can be used. 
- the frequence *freq* of acquisition in seconds. How often you want to fetch the data.

In [ ]:
from sylfenUtils.comUtils import ModbusDevice
dummy_device=ModbusDevice(ip='localhost',port=3500,device_name='dummy_device',
    dfplc=dummy_conf.dummy_df_plc,modbus_map=dummy_conf.dummy_modbus_map,bo='big',wo='big',freq=2)

## 2.2 Start the SIMULATOR of the modbus device (optionnal)
As we don't have the dummy device delivering data with the modbus protocole, we will create a simulator of the device based on the same modbus map and we will run the modbus server to serve data (random data). 
For your real project you can of course skip this step if the device is available on your network.

In [ ]:
from sylfenUtils.Simulators import SimulatorModeBus
dummy_simulator=SimulatorModeBus(port=dummy_device.port,modbus_map=dummy_device.modbus_map,bo=dummy_device.byte_order,wo=dummy_device.word_order)
dummy_simulator.start()

## 2.3 make sure you can connect to the device and COLLECT ALL THE DATA from your modbus device in real time. (not necessary in operation)

If you don't know the endianness you can instanciate your device anyway and yse the function **quick_modbus_single_register_decoder** with one register to guess the endianess.

In [ ]:
dummy_device.connectDevice()
dummy_device.quick_modbus_single_register_decoder(10,2,'float32',unit=1);

here we see that the only reasonable value is for the combination byte_order='big' and words_order='little because all the others have extrem values. If this is still not unambiguous after one call. You can call it a second time. 

In [ ]:
dummy_device.quick_modbus_single_register_decoder(10,2,'float32',unit=1);

now we can see that all the others combinations have their values changed too radically. So it is very very likely that the correct encoding is 'big', 'big'.

In [ ]:
tags=dummy_device.dfplc.index.to_list()
dummy_device.connectDevice()
data=dummy_device.collectData('CET',tags)## do not forget to precise the time zome
data

## 2.4 Check that you can INSERT DATA INTO THE DATABASE (not necessary in operation)
The database serves as a buffer for the realtime acquisition while simultaneously having the possibility to access the data. 
Check that the data were correclty inserted by retrieving all the data of your database.

In [ ]:
dummy_device.insert_intodb(dummy_conf.DB_PARAMETERS,dummy_conf.DB_TABLE,'CET',tags)

Check that your database was filled with the data

In [ ]:
dummy_conf.DB_PARAMETERS

In [ ]:
comUtils.read_db(dummy_conf.DB_PARAMETERS,dummy_conf.DB_TABLE) 

## 2.5 Instanciate your DUMPER
With your devices you are ready to start your dumper. You need to precise : 
- a dictionnary of the devices 
- the folder where the data will be stored.
- the parameters of the database 
- the time window of the database in seconds.
- the name of the table in the database *dbTable*
- the timezone used to record the data *tz_record*(by default 'CET')
- the filename of the log file(by default None)

In [ ]:
from sylfenUtils.comUtils import SuperDumper_daily
DEVICES = {
    'dummy_device':dummy_device,
}
log_file_name=os.path.join(dummy_conf.LOG_FOLDER,'dumper.log') ## give a name to your logger. 
log_file_name=None ## if you want either to have the information in the console use None.
dumper=SuperDumper_daily(DEVICES,dummy_conf.FOLDERPKL,dummy_conf.DB_PARAMETERS,dummy_conf.PARKING_TIME,
    dbTable=dummy_conf.DB_TABLE,tz_record=dummy_conf.TZ_RECORD,log_file=log_file_name)

## 2.6 Park the database first before starting to dump data in case the database is already big. 

In [ ]:
dumper.park_database()

## 2.7 start to dump data.

In [ ]:
dumper.start_dumping()

check that it is correctly feeding the database

In [ ]:
dumper.read_db() 

Remarks:
- opcua device are also available
- if you want to create a new device class that works neither with modbus nor with OPCUA protocole you can create a children class of comUtils.Device. Just make sure : 
    - to rewrite a function **collectData** that collect all the data from the plc dataframe of the device.
    - a function **connectDevice** that connects to the device.


# 3. READ THE DATA in REAL TIME
Now you can load your parked data to process and visualize them.

## 3.1 Instanciate the VISUALISER object
you need to enter almost all the same parameters as for the dumper. You may wonder and find it unconvenient because we could have included this object in the dumper which in that case you won't have to recreate an other object. The reasons for not having proceeded this way are :
- dissociation of the objects so that you can visualise data from a folder even if the dumper is not running or not working. Loading historical data is possible without the dumper. 
- it simplifies the architecture of the code. Modularity brings complexity but improve order, robustness, maintainability and readability.   

In [ ]:
from sylfenUtils.comUtils import VisualisationMaster_daily
cfg=VisualisationMaster_daily(
    dummy_conf.FOLDERPKL,
    dummy_conf.DB_PARAMETERS,
    dummy_conf.PARKING_TIME,
    dbTable=dummy_conf.DB_TABLE,
    tz_record=dummy_conf.TZ_RECORD
)

There is now a SMALL SHORTCOMING HERE OF THE LIBRARY. The visualiser needs the whole plc dataframe with all the tags of all your devices. 
You should not need to use the dumper to instanciate the visualiser. 
It would be better to concatenate the plc_dataframe of all your devices in one plc_dataframe.

In [ ]:
cfg.dfplc=dumper.dfplc ### required to use the function getTagsTU
cfg.listUnits=list(cfg.dfplc['UNITE'].unique())

## 3.2 Grab your tags 
if your plc dataframe is big you may need not remember all the tags ny heart and you would like to grab the tags with key expressions. 
Use the fonction **getTagsTU** of the object to look for tags using a regular expressions. 
For example we want to get all the pressure and temperature sensors that works with water(H2O) in our system.

In [ ]:
tags=cfg.getTagsTU('[PT]T.*H2O')
tags

## 3.3 LOAD the data precising the resampling method and resampling time. Here we want the last 2 hours
Of course you can get the data from anytime to anytime given t0,t1 as timestamps.

In [ ]:
t1=pd.Timestamp.now(tz='CET')
t0=t1-pd.Timedelta(hours=2)
df=cfg.loadtags_period(t0,t1,tags,rs='2s',rsMethod='mean')
df

## 3.4 PLOT the data with a standard multi unit scale graph
You can of course also plot the data with your own function. The advantages of this function is that it can plot the data using different y-scales.

In [ ]:
from sylfenUtils.utils import Graphics
Graphics().multiUnitGraph(df).show()

# 4. DEPLOY THE WEB INTERFACE
The interface enables other people(or you) to access the data in a convenient way from the web plateform.

## 4.1 PREPARE the folder of your app
For the application to work (with flask) it is needed to have a folder *templates/* and *static/* in the *root_folder*.

Chose your root folder 

In [ ]:
root_folder=os.path.join(dummy_conf.project_folder,'dashboard')
import importlib
import sylfenUtils.dashboard as dashboard
importlib.reload(dashboard)
root_folder

## 4.1 Configure the app with some default settings.

In [ ]:
init_parameters={
    'tags':cfg.getTagsTU('[PTF]T.*H2O'),
    'fig_name':'temperatures, pressures, and mass flows',
    'rs':'30s',
    'time_window':str(2*60),
    'delay_minutes':0,
    'log_versions':None #you can enter the relative path of (in folder static) a .md file summarizing some evolution in your code.
}

## 4.2 Instanciate the DASHBOARD

In [ ]:
APP_NAME='dummy_app'
dash=dashboard.Dashboard(
    cfg,
    dummy_conf.LOG_FOLDER,
    root_folder,
    app_name=APP_NAME,
    init_parameters=init_parameters,
    plot_function=cfg.utils.multiUnitGraph, ## you can use your own function to display the data 
    version_dashboard='1.0')

dash.helpmelink='' ### you can precise a url link on how to use the web interface
dash.fig_wh=780### size of the figure

In [ ]:
sylfenUtils.Conf_generator.__file__

If you need to extend some functionnality you can do it here.
For example you can ask the back end to print a text with front-end get '/example'

In [ ]:
@dash.app.route('/example', methods=['GET'])
def example_extension():
    print('this is a test')

## 4.4 Start the app

In [ ]:
port_app=15000
dash.app.run(host='0.0.0.0',port=port_app,debug=False,use_reloader=False)

- to deploy it to the world, open the port of the app on your routeur and use host='0.0.0.0'

As Flask says it is not a reliable as a production environment. It is better to use wsgi and gunicorn to run the app.
- more convenient is to control the process with with **systemctl** on a Linux server. 
- Then use **ngnix**(or apache) to make the redirection via a reverse proxy configured in a <myapp>.conf file in the nginx folders *sites_enabled* and *sites-available* to be able to map your domain name or subdomain name to the app. A guide is to be found [here](https://github.com/dorianSylfen/smallpower).